In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

Mounted at /content/drive


In [2]:
name_output = input('enter the name of your output file ').strip()

enter the name of your output file ensemble_1


# Import Modules

In [3]:
import tensorflow as tf

In [4]:
# import tensorflow as tf
# from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Reshape, Activation,Flatten,BatchNormalization,Conv2D,MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
%matplotlib inline
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Recall, Precision,TruePositives,TrueNegatives, FalsePositives,FalseNegatives,Accuracy

In [5]:
import os
import shutil
import glob

import random
import warnings

# Add GPU

In [6]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print('Num GPUs Available: ', len(physical_devices))
if len(physical_devices) >=1:
    tf.config.experimental.set_memory_growth(physical_devices[0],True)

Num GPUs Available:  0


# Make Your Edits

In [7]:
## change home directory
home = '/content/drive/MyDrive/Projects/CXR'

image_zip = 'chest_xray.zip'

model_name = 'ensemble'

this_model_name =  name_output+'.h5'

# Pull Data and Setup

In [8]:
image_path =  'data'

input_dir = 'train'
train_dir = 'data/train'
val_dir = 'data/validate'
held_dir = 'data/withhold'


shutil.copytree(os.path.join(home, 'data','data'), 'data')

'data'

In [ ]:
# prompt: check in data the number of items in both folders in train

normal_train_len = len(os.listdir('data/train/NORMAL'))
pneumonia_train_len = len(os.listdir('data/train/PNEUMONIA'))
print('NORMAL:', normal_train_len)
print('PNEUMONIA:', pneumonia_train_len)


NORMAL: 3216
PNEUMONIA: 3100


In [9]:
import glob

In [10]:
# prompt: specify a split value 0 to 1, move all data from data/withhold/NORMAL to data/train/NORMAL and data/evaluate/NORMAL based on the split value then do same for the PNEUMONIA class

split = 1 #@param {type:"slider", min:0, max:1, step:0.1}

for c in ['NORMAL','PNEUMONIA']:
  files = glob.glob(os.path.join('data','withhold',c,'*'))
  for f in files[:int(split*len(files))]:
    shutil.move(f, os.path.join('data','train',c))
  for f in files[int(split*len(files)):]:
    shutil.move(f, os.path.join('data','validate',c))


# Functions

### Plot Confusion Matrix

In [11]:
def plot_confusion_matrix(cm,classes,
                          normalize = False,
                          title='Confusion matrix',
                          cmap = plt.cm.Blues):
  """
  this function prints and plots the confusion matrix
  normalization can be applied by setting normalize=True
  """
  plt.imshow(cm,interpolation='nearest',cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tickmarks=  np.arange(len(classes))
  plt.xticks(tickmarks,classes,rotation=45)
  plt.yticks(tickmarks,classes)
  if normalize:
    cm = cm.astype('float')/cm.sum(axis=1) [:,np.newaxis]
    print('Normalized confusion matrix')
  else:
    print('confusion matrix without normalization')

  thresh = cm.max()/2
  for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
    plt.text(i,j,cm[i,j],
             horizontalalignment='center',
             color = 'white' if cm[i,j] > thresh else 'black'
             )
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('predicted label')

# Build Model

In [12]:
import os
import math
import numpy as np
import pickle as p
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import cv2
from tensorflow.random import set_seed
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import ResNet50



In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [14]:
model_dir = os.path.join(home,'models')

In [15]:
model_names = ['InceptionV3_softmax.h5', 'Vgg_with_sigmoid.h5','resnet_50_sig.h5','nasnet_sigmoid.h5','firstvgg.h5']

In [23]:
model_types = ['InceptionV3','VGG16','ResNet50','NasNet','VGG16']

In [17]:
# prompt: pull models to a list called base models

base_models = []
for i,m in enumerate(model_names):
  print(i,m)
  model = tf.keras.models.load_model(os.path.join(model_dir,m))
  for layer in model.layers:
    layer.trainable = False
  base_models.append((model))


0 InceptionV3_softmax.h5


1 Vgg_with_sigmoid.h5


2 resnet_50_sig.h5


3 nasnet_sigmoid.h5


4 firstvgg.h5


In [18]:
import numpy as np
from tensorflow.keras import layers, models
from sklearn.model_selection import KFold

class EnsembleMLP:
    def __init__(self, base_models, model_types, mlp_hidden_layers=[512, 256],
                 dropout_rate=0.5, output_units=1, output_activation='sigmoid', k_folds=5):
        """
        Initialize the EnsembleMLP class.

        Parameters:
        - base_models: List of pre-trained models to use as feature extractors.
        - model_types: List of model types (e.g., 'ResNet50', 'VGG16') corresponding to each base model.
        - mlp_hidden_layers: List of integers specifying the number of units in each hidden layer of the MLP.
        - dropout_rate: Float, dropout rate applied to each hidden layer in the MLP.
        - output_units: Integer, number of output units in the MLP.
        - output_activation: String, activation function for the output layer.
        - k_folds: Integer, number of folds for cross-validation when validation_batches is None.
        """
        self.base_models = base_models
        self.model_types = model_types
        self.k_folds = k_folds

        # Determine the total number of features from all base models
        total_features = sum([model.output_shape[-1] for model in self.base_models])

        # Create the MLP model
        mlp_input = layers.Input(shape=(total_features,))
        x = mlp_input

        # Add hidden layers
        for units in mlp_hidden_layers:
            x = layers.Dense(units, activation='relu')(x)
            x = layers.Dropout(dropout_rate)(x)

        # Output layer
        mlp_output = layers.Dense(output_units, activation=output_activation)(x)

        # Build the MLP model
        self.mlp_model = models.Model(mlp_input, mlp_output)

    def compile(self, **kwargs):
        """
        Compile the MLP model.

        Parameters:
        - kwargs: Keyword arguments to be passed to the compile method of the MLP model.
        """
        self.mlp_model.compile(**kwargs)

    def fit(self, X, y, validation_data=None, epochs=10, **kwargs):
        """
        Fit the MLP model using the concatenated predictions of the base models as input.

        Parameters:
        - X: Features from the base models' predictions.
        - y: Labels corresponding to the input features.
        - validation_data: Tuple (X_val, y_val), where X_val is the validation features and y_val is the validation labels.
        - epochs: Integer, number of epochs to train the model.
        - kwargs: Additional arguments for the fit method of the MLP model.
        """
        if validation_data is None:
            # Perform k-fold cross-validation
            kf = KFold(n_splits=self.k_folds, shuffle=True)

            for fold, (train_index, val_index) in enumerate(kf.split(X)):
                print(f"Training on fold {fold + 1}/{self.k_folds}...")
                X_train, X_val = X[train_index], X[val_index]
                y_train, y_val = y[train_index], y[val_index]

                self.mlp_model.fit(X_train, y_train, validation_data=(X_val, y_val),
                                   epochs=epochs // self.k_folds, **kwargs)
        else:
            # Use provided validation data
            self.mlp_model.fit(X, y, validation_data=validation_data, epochs=epochs, **kwargs)

    def predict(self, X):
        """
        Predict using the EnsembleMLP model.

        Parameters:
        - X: Test data (features).

        Returns:
        - Predictions made by the MLP model.
        """
        return self.mlp_model.predict(X)

    def load_data_from_directory(self, directory, batch_size=32, target_size=(224, 224), class_mode='binary'):
        """
        Load data from a directory using model-specific preprocessing functions.

        Parameters:
        - directory: Path to the data directory.
        - batch_size: Integer, batch size for the data generators.
        - target_size: Tuple, target size for the input images.
        - class_mode: String, mode for the class labels (e.g., 'binary', 'categorical').

        Returns:
        - X: Concatenated predictions from all base models.
        - y: Corresponding labels.
        """
        # Create data generators for each model type
        data_generators = []
        for model_type in self.model_types:
            data_gen = ImageDataGenerator(preprocessing_function=self._get_preprocessing_function(model_type))
            generator = data_gen.flow_from_directory(directory, target_size=target_size, batch_size=batch_size, class_mode=class_mode, shuffle=False)
            data_generators.append(generator)

        # Generate predictions and concatenate them
        concatenated_features = [] # each element of an array is one models prediction of all data
        # concatenated_targets = []
        y = None

        for i,gen in enumerate(data_generators):
            print(f'Gen {i} : model {model_types[i]} ')
            batch_features = []
            if y is None:

                batch_target = []

            for batch_X, batch_y in gen:
                features = self.base_models[i].predict( batch_X)

                batch_features.append(features)
                if y is None:
                    batch_target.append(batch_y)

                # if y is None:
                #     y = batch_y

                if len(batch_features) * batch_size >= gen.samples:
                    break

            concatenated_features.append(np.concatenate(batch_features, axis=0)) ## I want a vertical stack of each batch and each target
            if y is None:
                y = np.concatenate(batch_target, axis=0)


        X = np.concatenate(concatenated_features, axis=-1)
        return X, y

    def _get_preprocessing_function(self, model_type):
        """
        Get the appropriate preprocessing function for the specified model type.

        Parameters:
        - model_type: String, the type of model (e.g., 'ResNet50', 'VGG16').

        Returns:
        - Preprocessing function corresponding to the model type.
        """
        if model_type == 'ResNet50':
            from tensorflow.keras.applications.resnet50 import preprocess_input
        elif model_type == 'VGG16':
            from tensorflow.keras.applications.vgg16 import preprocess_input
        elif model_type == 'InceptionV3':
            from tensorflow.keras.applications.inception_v3 import preprocess_input
        elif model_type == 'DenseNet121':
            from tensorflow.keras.applications.densenet import preprocess_input
        elif model_type == 'NasNet':
            from tensorflow.keras.applications.nasnet import preprocess_input

        # Add more model types as needed
        else:
            raise ValueError(f"Unsupported model type: {model_type}")

        return preprocess_input


In [24]:
# prompt: initialize our ensemle model

ensemble_model = EnsembleMLP(base_models=base_models, model_types=model_types, mlp_hidden_layers=[512, 256])
ensemble_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:

# # Compile the MLP model
# ensemble_mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Fit the model using data generators with k-fold cross-validation
# ensemble_mlp.fit(train_batches=train_batches, epochs=10, steps_per_epoch=100)


In [ ]:
# prompt: ouput the plot of model to file in os.path.join(home,'Outputs','Pneunet.png)

# plot_model(model, to_file=os.path.join(home,'Outputs','atttn_resnet.png'), show_shapes=True, show_layer_names=True)


Output hidden; open in https://colab.research.google.com to view.

## Preprocess

# Pull data

In [21]:
train_path = os.path.join(image_path,'train')
valid_path = os.path.join(image_path,'validate')
test_path = os.path.join(image_path,'test')

In [25]:
# Pull data using the ensemble model's load_data_from_directory function
X_train, y_train = ensemble_model.load_data_from_directory(train_path)

Found 6839 images belonging to 2 classes.
Found 6839 images belonging to 2 classes.
Found 6839 images belonging to 2 classes.
Found 6839 images belonging to 2 classes.
Found 6839 images belonging to 2 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/ste

In [ ]:
X_train.shape

In [ ]:
pd.DataFrame(X_train)

In [ ]:
pd.DataFrame(y_train)

In [ ]:

# Convert to DataFrames
X_train_df = pd.DataFrame(X_train)
y_train_df = pd.DataFrame(y_train)

# Save to CSV
X_train_df.to_csv(os.path.join(home, 'data', 'ensemble_xtrain.csv'), index=False)
y_train_df.to_csv(os.path.join(home, 'data', 'ensemble_ytrain.csv'), index=False)


In [ ]:

X_val, y_val = ensemble_model.load_data_from_directory(valid_path)

In [ ]:
# pull data for the train test and validate using our ensemble model  data from directory function
X_test, y_test = ensemble_model.load_data_from_directory(test_path)


In [ ]:
# prompt: make X_train, _test xval yval  xtest and ytest as a dictionary  called eemble data and pickle the variablee move it to os.join(home,data)  so it could be used another time for faster prediction and training

ensemble_data = {
    'X_train': X_train,
    'X_test': X_test,
    'X_val': X_val,
    'y_val': y_val,
    'X_test': X_test,
    'y_test': y_test
}

# Save the dictionary to a pickle file
ensemble_data_path = os.path.join(home, 'data', 'ensemble_data.pkl')
with open(ensemble_data_path, 'wb') as f:
    p.dump(ensemble_data, f)

print(f"Ensemble data saved to {ensemble_data_path}")


Found 6316 images belonging to 2 classes.
Found 521 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


{'NORMAL': 0, 'PNEUMONIA': 1}


# Train the model

In [ ]:
# # Compile the Model
# model.compile(optimizer=Adam(learning_rate=0.0001),
#               loss='categorical_crossentropy',
#               metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [ ]:
epochs=  20

In [ ]:
# prompt: fit the ensemble model add the output to history

History = ensemble_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs)


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


In [ ]:
history_df = pd.DataFrame(History.history)

In [ ]:
history_df.to_csv(os.path.join(home,'Outputs',name_output+'.csv'))

# Optionally Load the Model

In [ ]:
# # prompt: Load a saved model and use

# from tensorflow.keras.models import load_model
# # Load the saved model
# model = load_model(os.path.join(home,'models',this_model_name))

# # Now you can use the loaded_model for predictions or further training
# # predictions = loaded_model.predict(test_batches)


# Predict

In [ ]:
# prompt: predict on X_test and  compare with y_test

y_pred = ensemble_model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

# Evaluate the performance
accuracy = metrics.accuracy_score(y_test, y_pred_binary)
precision = metrics.precision_score(y_test, y_pred_binary)
recall = metrics.recall_score(y_test, y_pred_binary)
f1_score = metrics.f1_score(y_test, y_pred_binary)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")


In [ ]:
# prompt: show sensitivity and specificity and then classification report as a dataframe

import pandas as pd
from sklearn.metrics import classification_report
# Calculate sensitivity and specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")

# Generate classification report
report = classification_report(y_test, y_pred_binary, output_dict=True)
report_df = pd.DataFrame(report).transpose()
print(report_df)


In [ ]:
predictions = model.predict(  x = x_test,verbose=0)

In [ ]:
dff = pd.DataFrame(predictions)

In [ ]:
dff['test'] = test_labels
dff.head(10)

# Taking Prediction Probabilities to CSV

In [ ]:
# dff.to_csv(os.path.join(home,'Outputs',name_output+'_prediction_probabilities.csv'))

# Confusion Matrix

In [ ]:
# prompt: create confusion metrric asd plot it with plot_confusion matrix formula

cm = confusion_matrix(y_test, y_pred_binary)
cm_plot_labels = ['NORMAL','PNEUMONIA']
plot_confusion_matrix(cm,cm_plot_labels,title='Confusion Matrix')


# Metrics

In [ ]:
# prompt: given y_pred and y_true calculate importatnt metrics to 4 dps

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')  # Use 'macro' for multiclass
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

# Print metrics to 4 decimal places
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


# Save Model

In [ ]:
model.save(this_model_name)

In [ ]:
model_path = os.path.join(home,'models')
# home = '/content/drive/MyDrive/project/Tensorflow And Keras' #/content/drive/MyDrive/project/Project2.1/Script2_pure classification

In [ ]:
shutil.move(this_model_name,model_path)